In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session


import matplotlib.pyplot as plt
import seaborn as sns
import missingno

In [ ]:
df_train = pd.read_csv('../input/hr-analytics-job-change-of-data-scientists/aug_train.csv')
df_test = pd.read_csv('../input/hr-analytics-job-change-of-data-scientists/aug_test.csv')

In [ ]:
df_train.head()

In [ ]:
df_test['target'] = -1 #remeber that we have to drop this column later

df_pre = pd.concat([df_train, df_test], axis = 0).reset_index(drop = True)

In [ ]:
df_pre.info()

In [ ]:
df_pre.columns

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
lbc=LabelEncoder()

In [ ]:
df_pre['city']=lbc.fit_transform(df_pre['city'])

In [ ]:
df_pre.head()

In [ ]:
df_lb=df_pre.copy()

In [ ]:
df_lb

In [ ]:
# Mapping ordinal categories
gender_map={'Male':1 ,
            'Female':2 ,
            'Other':0
}

relevent_experience_map = {
    'Has relevent experience':  1,
    'No relevent experience':    0
}

enrolled_university_map = {
    'no_enrollment'   :  0,
    'Full time course':    1, 
    'Part time course':    2 
}
    
education_level_map = {
    'Primary School' :    0,
    'Graduate'       :    2,
    'Masters'        :    3, 
    'High School'    :    1, 
    'Phd'            :    4
    } 
    
major_map ={ 
    'STEM'                   :    0,
    'Business Degree'        :    1, 
    'Arts'                   :    2, 
    'Humanities'             :    3, 
    'No Major'               :    4, 
    'Other'                  :    5 
}
    
experience_map = {
    '<1'      :    0,
    '1'       :    1, 
    '2'       :    2, 
    '3'       :    3, 
    '4'       :    4, 
    '5'       :    5,
    '6'       :    6,
    '7'       :    7,
    '8'       :    8, 
    '9'       :    9, 
    '10'      :    10, 
    '11'      :    11,
    '12'      :    12,
    '13'      :    13, 
    '14'      :    14, 
    '15'      :    15, 
    '16'      :    16,
    '17'      :    17,
    '18'      :    18,
    '19'      :    19, 
    '20'      :    20, 
    '>20'     :    21
} 
    
company_type_map = {
    'Pvt Ltd'               :    0,
    'Funded Startup'        :    1, 
    'Early Stage Startup'   :    2, 
    'Other'                 :    3, 
    'Public Sector'         :    4, 
    'NGO'                   :    5
}

company_size_map = {
    '<10'          :    0,
    '10/49'        :    1, 
    '100-500'      :    2, 
    '1000-4999'    :    3, 
    '10000+'       :    4, 
    '50-99'        :    5, 
    '500-999'      :    6, 
    '5000-9999'    :    7
}
    
last_new_job_map = {
    'never'        :    0,
    '1'            :    1, 
    '2'            :    2, 
    '3'            :    3, 
    '4'            :    4, 
    '>4'           :    5
}

In [ ]:
df_lb.loc[:,'relevent_experience']=df_lb['relevent_experience'].map(relevent_experience_map)
df_lb.loc[:,'enrolled_university']=df_lb['enrolled_university'].map(enrolled_university_map)
df_lb.loc[:,'education_level']=df_lb['education_level'].map(education_level_map)
df_lb.loc[:,'major_discipline']=df_lb['major_discipline'].map(major_map)
df_lb.loc[:,'company_size']=df_lb['company_size'].map(company_size_map)
df_lb.loc[:,'company_type']=df_lb['company_type'].map(company_type_map)
df_lb.loc[:,'last_new_job']=df_lb['last_new_job'].map(last_new_job_map)
df_lb.loc[:,'experience']=df_lb['experience'].map(experience_map)
df_lb.loc[:,'gender']=df_lb['gender'].map(gender_map)

In [ ]:
df_lb.info()

In [ ]:
missingno.bar(df_lb,color='dodgerblue',sort='ascending',figsize=(10,5),fontsize=12)


k-Nearest Neighbour Imputation


In [ ]:
df_lb1=df_lb.copy()

In [ ]:
## K means clustering to impute missing values

from sklearn.impute import KNNImputer
import sklearn.metrics as metrics

In [ ]:
knn_imputer=KNNImputer(n_neighbors=3)

In [ ]:
missing_cols = df_lb1.columns[df_lb1.isna().any()].tolist()
missing_cols

In [ ]:
#dataframe having features with missing values
df_missing = df_lb1[['enrollee_id'] + missing_cols]

#dataframe having features without missing values
df_non_missing = df_lb1.drop(missing_cols, axis = 1)

In [ ]:
df_non_missing

In [ ]:
df_lb1.info()

In [ ]:
X=knn_imputer.fit_transform(df_missing)

In [ ]:
df_missing=pd.DataFrame(X,columns=df_missing.columns)
df_missing.head()

In [ ]:
df_lb2=pd.merge(df_missing,df_non_missing,on='enrollee_id')

In [ ]:
test=df_lb2[df_lb2['target']==-1].reset_index(drop=True)
train=df_lb2[df_lb2['target']!=-1].reset_index(drop=True)

In [ ]:
X=train.drop(['enrollee_id','target'],axis=1)
y=train['target']
test=test.drop('target',axis=1)

 Modeling with Decision Tree Classifier



In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

In [ ]:
train_X,test_X,train_y,test_y=train_test_split(X,y,test_size=0.2,random_state=446)

In [ ]:
model=DecisionTreeClassifier(max_depth= 7,
 max_leaf_nodes= 11)# Parameters by Grid search hyper parameters look below for the best parameters

model.fit(train_X,train_y)

In [ ]:
# Hyper Parameters

param_grid={ 'max_depth':[2,3,4,5,6,7,8,9,10,11,12],
           'max_leaf_nodes':[2,3,4,5,6,7,8,9,10,11,12],
           }


In [ ]:
model=GridSearchCV(DecisionTreeClassifier(),param_grid,cv=5,scoring='f1_weighted')
model.fit(train_X,train_y)

In [ ]:
model.best_params_

In [ ]:
Train_AUC_score=metrics.roc_auc_score(train_y,pd.DataFrame(model.predict_proba(train_X))[1])
Test_AUC_score=metrics.roc_auc_score(test_y,pd.DataFrame(model.predict_proba(test_X))[1])

In [ ]:
print(f"Train AUC Score {Train_AUC_score}")
print(f"Test AUC Score {Test_AUC_score}")

In [ ]:
# Plot AUC CUrve

def plot_auc_curve(fpr, tpr, auc):
    plt.figure(figsize = (16,6))
    plt.plot(fpr,tpr,'b+',linestyle = '-')
    plt.fill_between(fpr, tpr, alpha = 0.5)
    plt.ylabel('True Postive Rate')
    plt.xlabel('False Postive Rate')
    plt.title(f'ROC Curve Having AUC = {Test_AUC_score}')

In [ ]:
fpr,tpr,_=metrics.roc_curve(test_y,pd.DataFrame(model.predict_proba(test_X))[1])

In [ ]:
plot_auc_curve(fpr, tpr, Test_AUC_score)